In [2]:
from utilities import ListRunsParser
from ipywidgets import widgets


Example 1

In [3]:
o_parser = ListRunsParser(current_runs="1,2,4:15")
o_parser.list_current_runs
new_runs = ['6','8', '10']
o_parser.new_runs(list_runs=new_runs)

'1,2,4,5,7,9,11:15'

Example 1 (continued)

In [4]:
new_runs = ['6','8', '10']
o_parser.new_runs(list_runs=new_runs)

'1,2,4:15'

Example 2

In [5]:
o_parser = ListRunsParser(current_runs="1,2,3,10")
o_parser.list_current_runs
new_runs = ['6']
o_parser.new_runs(list_runs=new_runs)

'1:3,6,10'

Example 3

In [6]:
o_parser = ListRunsParser(current_runs="")
o_parser.list_current_runs
new_runs = ['6','8','10','11','12']
print(o_parser.new_runs(list_runs=new_runs))

6,8,10:12


Example 4

In [7]:
o_parser = ListRunsParser(current_runs="")
o_parser.list_current_runs
new_runs = ['6']
print(o_parser.new_runs(list_runs=new_runs))
print(o_parser.list_current_runs)

6
['6']


In [8]:
def status_changed(selection):
    new_run_selected = selection['new']

list_runs = ['1','10','20','30','40','50','1','2','3']
_runs = ",".join(list_runs)
text = widgets.Text(value=_runs)
display(text)
ui = widgets.SelectMultiple(options=list_runs,
                           disabled=True)
display(ui)
ui.observe(status_changed, 'value')

# let's start with all the runs selected
list_runs_selected = {}
for _run in list_runs:
    list_runs_selected[_run] = True

Text(value='1,10,20,30,40,50,1,2,3')

SelectMultiple(disabled=True, options=('1', '10', '20', '30', '40', '50', '1', '2', '3'), value=())

In [9]:
ui.index = (0,1,2)

Full UI test to simulate real use of class

In [10]:
## widgets event handler

full_list_of_runs_selected = []
list_of_runs_already_selected = []
run_numbers_text_ui = None
run_numbers_error_ui = None

o_parser = ListRunsParser(current_runs="")
#o_parser.list_current_runs
#new_runs = ['6','8', '10']
#o_parser.new_runs(list_runs=new_runs)


def run_numbers_text_changed(value):
    global selected_runs_ui
    global run_numbers_error_ui
    global list_runs
    global last_row_ui
    
    new_text = value['new']
    _error_label_visibility = 'hidden'
    list_current_runs = []
    try:
        o_parser = ListRunsParser(current_runs=new_text)
        list_current_runs = o_parser.list_current_runs
        selected_runs_ui.options = list_current_runs
    except ValueError:
        _error_label_visibility = 'visible'
    
    # check runs that do not exist in full_list_of_runs
    list_runs_do_not_exist = []
    for _run in list_current_runs:
        if _run in list_runs:
            continue
        list_runs_do_not_exist.append(_run)

    if list_runs_do_not_exist:
        last_row_ui.layout.visibility = 'visible'
        last_row_ui.children[1].value = ",".join(list_runs_do_not_exist)
    else:
        last_row_ui.layout.visibility = 'hidden'
        
    
    run_numbers_error_ui.layout.visibility = _error_label_visibility
    
def full_list_of_runs_changed(value):
    global full_list_of_runs_selected
    full_list_of_runs_selected = value['new']
    
def selected_runs_ui_changed(value):
    pass
    
def add_button_clicked(value):
    global full_list_of_runs_selected
    global list_of_runs_already_selected
    global selected_runs_ui
    global run_numbers_text_ui

    for _run in full_list_of_runs_selected:
        list_of_runs_already_selected.append(_run)
        
    list_of_runs_already_selected = list(set(list_of_runs_already_selected))
    list_of_runs_already_selected.sort()
    
    selected_runs_ui.options = list_of_runs_already_selected
    
    o_parser = ListRunsParser()
    new_str_runs = o_parser.new_runs(list_of_runs_already_selected)
    print(new_str_runs)
    
    run_numbers_text_ui.value = new_str_runs
    
#     print("full_list_of_runs_selected: {}".format(full_list_of_runs_selected))
#     print("list_of_runs_already_selected: {}".format(list_of_runs_already_selected))

def remove_button_clicked(value):
    global selected_runs_ui
    global list_of_runs_already_selected
    
    list_of_runs_to_remove = set(selected_runs_ui.value)
    list_of_runs_already_selected = set(list_of_runs_already_selected)
    
    new_list_of_runs_already_selected = list_of_runs_already_selected - list_of_runs_to_remove
    
    selected_runs_ui.options = new_list_of_runs_already_selected
    list_of_runs_already_selected = list(new_list_of_runs_already_selected)

    str_list_of_runs_already_selected = ",".join(list_of_runs_already_selected)
    
    if len(list_of_runs_already_selected) > 1:
        o_parser = ListRunsParser(current_runs=str_list_of_runs_already_selected)
        new_str_runs = o_parser.new_runs()
    elif len(list_of_runs_already_selected) == 1:
        new_str_runs = str(list_of_runs_already_selected[0])
    else:
        new_str_runs = ''

    run_numbers_text_ui.value = new_str_runs

## initialize

import numpy as np
int_list_runs = np.arange(30) + 2000
list_runs = [str(_run) for _run in int_list_runs]
    
## build UI

first_row_ui = widgets.HBox([widgets.Label("Run numbers:",
                                           layout=widgets.Layout(width='10%')),
                           widgets.Text("",
                                        placeholder='',
                                       layout=widgets.Layout(width='40%')),
                            widgets.Label(">> Format Error! <<",
                                         layout = widgets.Layout(visibility='hidden',
                                                                 color='red'))])
first_row_ui.children[1].observe(run_numbers_text_changed, 'value')
run_numbers_text_ui = first_row_ui.children[1]
run_numbers_error_ui = first_row_ui.children[2]

col_1_ui = widgets.VBox([widgets.Label("Full list of runs"),
                        widgets.SelectMultiple(options=list_runs,
                                              layout=widgets.Layout(height='150px',
                                                                   width='100px'))],
                       layout=widgets.Layout(width='120px'))
col_1_ui.children[1].observe(full_list_of_runs_changed, 'value')

col_2_ui = widgets.VBox([widgets.Button(description="Add ->",
                                       layout=widgets.Layout(border='1px solid blue',
                                                               width='120px'))],
                       layout=widgets.Layout(width='165px',
                                            align_self='center',
                                            margin='10px'))
col_2_ui.children[0].on_click(add_button_clicked)

col_3_ui = widgets.VBox([widgets.Label("Selected runs"),
                        widgets.SelectMultiple(options=[],
                                              layout=widgets.Layout(height='150px',
                                                                   width='100px'))],
                       layout=widgets.Layout(width='120px'))
col_3_ui.children[1].observe(selected_runs_ui_changed, 'value')
selected_runs_ui = col_3_ui.children[1]

col_4_ui = widgets.Button(description="Remove",
                          layout=widgets.Layout(width='120px',
                                                border='1px solid red',
                                                margin='5px',
                                                align_self='flex-end'))
col_4_ui.on_click(remove_button_clicked)

second_row_ui = widgets.HBox([col_1_ui, col_2_ui, col_3_ui, col_4_ui],
                            layout=widgets.Layout(border='1px solid lightgrey',
                                                 padding='3px',
                                                 margin='5px 5px 5px 5px',
                                                 width='50%'))


last_row_ui = widgets.HBox([widgets.Label("Invalid run(s):",
                                         layout=widgets.Layout(width='8%')),
                            widgets.Label("",
                                        layout=widgets.Layout(width='80%'))],
                          layout=widgets.Layout(visibility='hidden'))

layout_ui = widgets.VBox([first_row_ui, second_row_ui, last_row_ui])

display(layout_ui)





VBox(children=(HBox(children=(Label(value='Run numbers:', layout=Layout(width='10%')), Text(value='', layout=Layout(width='40%'), placeholder=''), Label(value='>> Format Error! <<', layout=Layout(visibility='hidden')))), HBox(children=(VBox(children=(Label(value='Full list of runs'), SelectMultiple(layout=Layout(height='150px', width='100px'), options=('2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025', '2026', '2027', '2028', '2029'), value=())), layout=Layout(width='120px')), VBox(children=(Button(description='Add ->', layout=Layout(border='1px solid blue', width='120px'), style=ButtonStyle()),), layout=Layout(align_self='center', margin='10px', width='165px')), VBox(children=(Label(value='Selected runs'), SelectMultiple(layout=Layout(height='150px', width='100px'), options=(), value=())), layout=Layout(width='120px')), Button(description='Remove', layout=Layout(align_self='flex-end', border='1px solid red', margin='5px', width='120px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 5px 5px 5px', padding='3px', width='50%')), HBox(children=(Label(value='Invalid run(s):', layout=Layout(width='8%')), Label(value='', layout=Layout(width='80%'))), layout=Layout(visibility='hidden'))))

2001:2005
